In [130]:
import numpy as np
import matplotlib.pyplot as plt

report = ''
with open('report.txt', 'r') as f:
    report = f.read().split('\n')

q_set = set()
p_set = set()
d_set = set()
c_set = set()

for line in report:
    if line == '':
        continue
    cat = line.split('_')
    q_set.add(cat[1])
    p_set.add(cat[2])
    d_set.add(cat[3])
    c_set.add(cat[4].split(';')[0])

In [ ]:

hc_param = list(map(lambda x: x[1:], list(p_set)))
hc_param.reverse()
hc_suffix = 'producers'
hc_len = len(hc_param)

fig, axs = plt.subplots(2, hc_len)
print(axs)
for i in range(0, hc_len):
    axs[0, i].set_title(f'{hc_param[i]} {hc_suffix}')
    for j in range(1, len(axs)):
        axs[j, i].set(yticklabels=[])
    

# ax: rect
# cat: categories for bar chart
# ys: bars per category 
def plot_bars(ax, cat, ys):
    width = 0.25
    x = np.arange(len(cat))
    for i in range(len(ys)):
        # 0 1 2  - 
        ax.bar(x + (i - len(ys)/2) * width + width / 2, ys[i], width)
        ax.set_xticks(x, cat)
    
# turn into function
# parameters
ax = axs[0, 0]
cat = list(q_set)
cat.reverse()
ys = [np.random.rand(len(cat)), np.random.rand(len(cat))]

plot_bars(ax, cat, ys)